In [3]:
from flask import Flask
from flask import request
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

# server = Flask(__name__)
# app = dash.Dash(
#     __name__,
#     server = server,
#     serve_locally = False,
#     requests_pathname_prefix = "/plotary/dash/",
# )

app = dash.Dash(__name__,url_base_pathname="/plotary/dash/")

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
        html.Div([
#         html.A([
#             html.Div(['goto man-es page'],id="logo")
#         ],href='https://man-es.com'),
        html.Div([
            html.H1('Keine Auswertung ausgewählt. ')
        ],id="description"),
        # This Link does not seem to be causing an invocation of the callback 
        dcc.Link('Navigate to https://man-es.com', href='https://man-es.com'),
        html.Br(),
        # These 2 Links DO seem to be causing an invocation of the callback 
        dcc.Link('Navigate to "/plotary/dash/"', href='/plotary/dash/'),
        html.Br(),
        dcc.Link('Navigate to "/plotary/"', href='/plotary/'),
            
    ],
    id="navbar")]
)

@app.callback(Output('description', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    print(f'entering callback for pathname {pathname}')
    return html.Div([
        html.H1('Auswertung Nr {}'.format(pathname))
    ])

# if __name__ == '__main__':
#     app.run_server(port=8882,debug=False)

In [32]:
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import dash_table

def make_dt(table_id,df,display_headers=True):
    dt = dash_table.DataTable(
        id=table_id,
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="custom",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
#         style_cell={
#             'width': '{}%'.format(len(df.columns)),
#             'textOverflow': 'ellipsis',
#             'overflow': 'hidden'
#         }
    )
    if not display_headers:
        dt.style_header = {'display': 'none'}        
    return dt

    
def make_text_centered_div(text):    
    col_inner_style = {
        'margin':'auto',
        'word-break':'break-all',
        'word-wrap': 'break-word',
        'text-align':'center'
    }
    return html.Div([text],style=col_inner_style)

# make simple DataFrame
x = np.arange(0,20)
y = x**2
df = pd.DataFrame({'x':x,'y':y})

# create an "aggregates" DataFrame
df_agg = pd.DataFrame({'x':[int(df.x.mean())],'y':[df.y.sum()]})

# create dash_tables
dt = make_dt('mydt',df)
dt_agg = make_dt('mydt_agg',df_agg,display_headers=False)
agg_title = make_text_centered_div("aggregates")

# make app
app = dash.Dash(__name__)
app.layout = html.Div([dt,agg_title,dt_agg])
app.title = "My DataFrame With Aggs"

@app.callback(
    Output('mydt_agg','data'),
    [Input('mydt', 'rows'),
     Input('mydt', 'selected_row_indices')])
def selected_callback(rows,selected_row_indices):
    try:
        df_selected_rows=pd.DataFrame(rows)
        df_new_agg = pd.DataFrame({'x':[df_selected_rows.x.mean()],'y':[df_selected_rows.y.sum()]})
        d = df_new_agg.to_dict('records')
        return d
    except Exception as e:
        print(str(e))
        print(rows,selected_row_indices)
        raise PreventUpdate()

@app.callback(
    Output('mydt','data'),
#     [Input('mydt','filter_query')]
    [Input('mydt','derived_filter_query_structure')]
)
def process_filter(filter_query):
    print(filter_query,type(filter_query))
    return df.to_dict('records')
    if filter_query is None or len(filter_query)<=0:
        return df.to_dict('records')
    df_temp = df.copy()
    #{x} > 10 && {y} > 50
    fq = str(filter_query).replace('"','').replace('{',"").replace('}',"").replace('&&','and')
    print(fq)
    df_temp = df_temp.query(fq)
    return df_temp.to_dict('records')


app.run_server(host='127.0.0.1',port=8884)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8884/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jul/2020 16:20:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "POST /_dash-update-component HTTP/1.1" 200 -


None <class 'NoneType'>
'DataFrame' object has no attribute 'x'
None None
None <class 'NoneType'>


127.0.0.1 - - [15/Jul/2020 16:20:58] "POST /_dash-update-component HTTP/1.1" 200 -


{'subType': '>=', 'type': 'relational-operator', 'value': '>=', 'left': {'subType': 'field', 'type': 'expression', 'value': 'x'}, 'right': {'subType': 'value', 'type': 'expression', 'value': 10}} <class 'dict'>


127.0.0.1 - - [15/Jul/2020 16:22:18] "POST /_dash-update-component HTTP/1.1" 200 -


{'subType': '>=', 'type': 'relational-operator', 'value': '>=', 'left': {'subType': 'field', 'type': 'expression', 'value': 'x'}, 'right': {'subType': 'value', 'type': 'expression', 'value': '10 and <=22'}} <class 'dict'>
